- to calculate bayesian rule with the height of the hip;

In [16]:
import cv2
import mediapipe as mp
import time

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose()


prev_y = None
fall_threshold = 5
fall_detected = False


# to calculate prior probability;
p_E = 0.5
p_H = 0.5

p_nE = 1 - p_E
p_nH = 1 - p_H


# likelihood;
p_E_H = 0.3
p_E_nH = 0.4

#to calculate posteria probability;
p_H_E = (p_E_H * p_H) / (p_E_H * p_H + p_E_nH * p_nH) 
print(f'----- test code; calculate posteria; {p_H_E}')




cap = cv2.VideoCapture('fall.mp4')

output_path = 'fall_detection.avi'
out = None



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


        
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        y = int(left_hip.y * frame.shape[0])

        
        #to define falls if dy is less threshold;
        if prev_y is not None:
            dy = y - prev_y

            if dy > fall_threshold:
                fall_detected = True

                #to print the pixel number of dy;
                print(f'pixel of dy; {dy}')



                # likelihood;
                p_E_H = 0.7
                p_E_nH = 0.2
                
                #to calculate posteria probability;
                p_H_E = (p_E_H * p_H) / (p_E_H * p_H + p_E_nH * p_nH) 
                print(f'----- calculate posteria; {p_H_E}')
                


                
                
        prev_y = y


    
    if fall_detected:
        cv2.putText(frame, 'Fall Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0,0,255), 3)

    if out is None:
        height, width, _ = frame.shape
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 10, (width, height))

    out.write(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if out:
    out.release()
cv2.destroyAllWindows()

----- test code; calculate posteria; 0.4285714285714286
pixel of dy; 7
----- calculate posteria; 0.7777777777777778
pixel of dy; 9
----- calculate posteria; 0.7777777777777778
pixel of dy; 6
----- calculate posteria; 0.7777777777777778
pixel of dy; 8
----- calculate posteria; 0.7777777777777778
pixel of dy; 8
----- calculate posteria; 0.7777777777777778
pixel of dy; 6
----- calculate posteria; 0.7777777777777778
pixel of dy; 11
----- calculate posteria; 0.7777777777777778
pixel of dy; 21
----- calculate posteria; 0.7777777777777778
pixel of dy; 18
----- calculate posteria; 0.7777777777777778
pixel of dy; 23
----- calculate posteria; 0.7777777777777778
pixel of dy; 13
----- calculate posteria; 0.7777777777777778
pixel of dy; 14
----- calculate posteria; 0.7777777777777778
pixel of dy; 6
----- calculate posteria; 0.7777777777777778
pixel of dy; 7
----- calculate posteria; 0.7777777777777778


- for multiple case
- dy가 구간(, < 5 pixel, < 10 pixel, < 15pixel)에 따라 다른 likelihood를 주고, 구간별 낙상확률을 계산하라

- cf, certainty factor를 계산하라